In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import torch 
from torchvision import transforms, utils
from torch.utils.data import Dataset
from skimage import io, transform
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans2, whiten


from torchvision import models
from pytorch_model_summary import summary

In [2]:
# Enable autoreloading of imported modules.
%load_ext autoreload
%autoreload 2

In [3]:
torch.cuda.is_available()

True

In [4]:
def get_lat_lon(rel_path_to_images):
    """
    Extracts lat and lon from image name and store them in array
    !! Co-ordinates are saved as [Lon, Lat] !!
    """

    # get list of all filenames (insert path as string)
    file_names = os.listdir(rel_path_to_images)

    # strip everything but the coordinates and split lattitude and longitude
    coord = [i[4:-4] for i in file_names]

    # split to get lat and long and change type to np array 
    labels = np.array([np.array(i.split(","), dtype = float) for i in coord])
    
    return labels

In [5]:
#ToDo: Find optimal k (use Elbow Method?)
# Different outputs here because i was playing around with formats for the dataloader and thought
# we could eventually use those later

def get_clusters(labels, k):

    """
    Create k clusters and assign a cluster to each sample
        
        Returns:
        labels = array shape N*3 of sample co-ordinates with their respective cluster
        label_location: Dict that contains the center co-ordinates of each cluster
        
    """
    
    x, y = kmeans2(labels, k, iter = 20)  
    labels = np.hstack((labels, y[:, np.newaxis]))
    
    # create dict to map class to co-ordinates for final prediction
    keys = np.arange(len(x))
    label_location = {keys[i]:x[i] for i in range(len(x))}
    
    return labels, label_location, y

In [6]:
# create Dataloader
# in the NN we would only predict classes, not yet coordinates. This allows y to be one number per sample
# This can later be changed, just seemed more convenient to me

class GeoGuessrData(Dataset):
    
    def __init__(self, y, root_dir, transform = None):
        """
        y(array): array shape N with clusternames
        root_dir(string): Directory with all the images
        transform: As we won´t need to resize anything we just transform array-->tensor
        """
        self.y = y
        self.root_dir = root_dir
        self.transform = transform
        
    def __getitem__(self, idx):
        """support the indexing such that dataset[i] can be used to get ith sample"""
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # get image by path and idx
        img_name = os.listdir(self.root_dir)[idx]
        image = io.imread(os.path.join(self.root_dir, img_name))
        sample = {"image": image, "cluster": self.y[idx]}
        
        if self.transform:
            sample["image"] = self.transform(sample["image"])
            
        return sample
        
    
    def __len__(self):
        
        # get len of dataset
        return len(self.y)
    
    

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, cluster = sample['image'], sample['cluster']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = image.transpose((2, 0, 1))
        
        # not 100% sure if transforming y is of any use yet
        return {'image': torch.from_numpy(image).float(),
                'cluster': torch.from_numpy(np.array(cluster)).float()}

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])
# load dataset for dataloader
labels = get_lat_lon(r"C:\Users\Shadow\Pictures\Geogussr\Projekt")
labels, label_location, y = get_clusters(labels, 100)
dataset = GeoGuessrData(y, r"C:\Users\Shadow\Pictures\Geogussr\Projekt", transform=transform)




C:\Users\Shadow\.conda\envs\pytorch\lib\site-packages\scipy\cluster\vq.py:603: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


In [8]:
size = len(dataset)
train_size = int(size*0.8)
test_size = (size - train_size) //2
validation_size = (size - train_size) //2 +1
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset,
                                               [train_size, validation_size, test_size])


In [9]:
from networks import ResNet
from networks import GeoGuessrNet

HEIGHT = 512
WIDTH = 2560

model=GeoGuessrNet()
#summary(model, (3,HEIGHT, WIDTH))

In [10]:

from solver import Solver

torch.cuda.empty_cache()

solver = Solver(model,{'train':test_dataset,'val':validation_dataset},optimizer="Adam",optimizer_config={'weight_decay':True})
history =solver.train(num_epochs=10)


cuda
train, we are stuck
train, we are not stuck!


  0%|          | 0/10 [00:00<?, ?it/s]

oh sole mio
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
test, we are stuck
test, we are not stuck!


AttributeError: 'str' object has no attribute 'to'

In [1]:
from utils import show_training
show_training(history)

NameError: name 'history' is not defined

In [ ]:
import gc
torch.cuda.empty_cache()
#del variables
gc.collect()

In [ ]:
dataloader = torch.utils.data.DataLoader(
            self.data_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True          
        )